# Initiate and configure Spark Session and Context

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .config("spark.driver.memory", "12g")\
    .appName("review_city_neighbourhood") \
    .getOrCreate()

sc = spark.sparkContext

print("Using Apache Spark Version", spark.version)
web_ui_url = sc.uiWebUrl
print(f"Spark UI is available at: {web_ui_url}")

24/04/27 14:33:55 WARN Utils: Your hostname, Kun-Mac.local resolves to a loopback address: 127.0.0.1; using 10.206.101.44 instead (on interface en0)
24/04/27 14:33:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/27 14:33:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Using Apache Spark Version 3.5.0
Spark UI is available at: http://10.206.101.44:4040


# Read CSV into Spark Dataframe

In [2]:
listings = spark.read.option("header", "true") \
                   .option("delimiter", ",") \
                   .option("inferSchema", "true") \
                   .option("multiLine", "true")\
                   .option("escape", "\"")\
                   .csv("../Data/listings.csv")
listings.printSchema()

root
 |-- id: long (nullable = true)
 |-- listing_url: string (nullable = true)
 |-- scrape_id: long (nullable = true)
 |-- last_scraped: date (nullable = true)
 |-- source: string (nullable = true)
 |-- name: string (nullable = true)
 |-- description: string (nullable = true)
 |-- neighborhood_overview: string (nullable = true)
 |-- picture_url: string (nullable = true)
 |-- host_id: integer (nullable = true)
 |-- host_url: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- host_since: date (nullable = true)
 |-- host_location: string (nullable = true)
 |-- host_about: string (nullable = true)
 |-- host_response_time: string (nullable = true)
 |-- host_response_rate: string (nullable = true)
 |-- host_acceptance_rate: string (nullable = true)
 |-- host_is_superhost: string (nullable = true)
 |-- host_thumbnail_url: string (nullable = true)
 |-- host_picture_url: string (nullable = true)
 |-- host_neighbourhood: string (nullable = true)
 |-- host_listings_count: dou

In [3]:
# calendar = spark.read.option("header", "true") \
#                    .option("delimiter", ",") \
#                    .option("inferSchema", "true") \
#                    .option("multiLine", "true")\
#                    .option("escape", "\"")\
#                    .csv("../Data/calendar.csv")
# calendar = calendar.drop(*['minimum_nights','maximum_nights'])
# #calendar.show()
# calendar.printSchema()

**city**

In [4]:
listings.createOrReplaceTempView('listings')
# calendar.createOrReplaceTempView('calendar')
city_query = '''
SELECT
    l.city,
    l.state,
    COUNT(DISTINCT l.id) AS total_listings
FROM listings l
GROUP BY 
    l.city
    ,l.state
-- ORDER BY l.city
'''

result_city = spark.sql(city_query)

24/04/27 14:34:01 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [5]:
import pandas
result_city.limit(5).toPandas()

,city,state,total_listings
0,nashville,tn,8850
1,broward-county,fl,18230
2,new-york-city,ny,39202
3,boston,ma,4204
4,austin,tx,15419


**neighborhood**

In [6]:
# listings.createOrReplaceTempView('listings')
# calendar.createOrReplaceTempView('calendar')

neighbourhood_query = '''
SELECT
    l.neighbourhood_cleansed,
    l.city,
    l.state,
    COUNT(DISTINCT l.id) AS total_listings
FROM listings l
GROUP BY
    l.neighbourhood_cleansed,
    l.city,
    l.state
'''

result_neighbourhood = spark.sql(neighbourhood_query)

In [7]:
result_neighbourhood.limit(5).toPandas()

24/04/27 14:34:07 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


,neighbourhood_cleansed,city,state,total_listings
0,Woodland Hills,los-angeles,ca,525
1,Kips Bay,new-york-city,ny,417
2,Upper Laurel,oakland,ca,28
3,Overlook,portland,or,122
4,Lake Los Angeles,los-angeles,ca,10
